In [103]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize, sent_tokenize
import os
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3]:
paragraphs = pd.read_csv("data/share_repurchase_paragraphs.csv")

In [4]:
paragraphs.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ..."
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ..."
4,A,0001090872-17-000018,Amount Spent on Share Repurchases,repurchased,194000000,194.00,millions,"Table of Contents 2016, upon the completion ..."


In [6]:
paragraphs.paragraph_text[0]

'On May 28, 2015 we  announced that our board  of directors had approved a  new share repurchase program  (the   "2015 repurchase program"). The 2015 share repurchase program authorizes the purchase of up to $1.14  billion   of our common stock at the company\'s discretion  through and including November 1, 2018. The 2015  repurchase   program does  not require  the company  to  acquire a  specific number  of  shares and  may be  suspended  or   discontinued at any time. During the year ended October 31,                                                        95'

In [90]:
test_p = paragraphs.paragraph_text[2].lower()

In [91]:
test_p

'table of contents   2016, upon the completion of our previous repurchase program, we repurchased approximately 8.3 million shares   for $336  million  under  this  authorization.  during  the  year  ended  october  31,  2017  we  repurchased   approximately 4.1 million shares for $194  million under this authorization. as  of october 31, 2017, we  had   remaining authorization to repurchase up to $610 million of our common stock under this program'

In [92]:
paragraphs.text[1]

'The 2015 share repurchase program authorizes the purchase of up to'

for root, dirs, files in os.walk("data/nc_training_filings", topdown=True):
    for file in files:
        print(file)
        # df = pd.read_csv(os.path.join(root, file), encoding = "ISO-8859-1", low_memory = False)
        # nih_full = nih_full.append(df)

In [93]:
test_p_sent = sent_tokenize(test_p)

In [94]:
test_p_sent

['table of contents   2016, upon the completion of our previous repurchase program, we repurchased approximately 8.3 million shares   for $336  million  under  this  authorization.',
 'during  the  year  ended  october  31,  2017  we  repurchased   approximately 4.1 million shares for $194  million under this authorization.',
 'as  of october 31, 2017, we  had   remaining authorization to repurchase up to $610 million of our common stock under this program']

In [95]:
test_p_sent_bow = list()
for sent in test_p_sent:
    test_p_sent_bow.append(word_tokenize(sent))

In [96]:
test_p_sent_bow

[['table',
  'of',
  'contents',
  '2016',
  ',',
  'upon',
  'the',
  'completion',
  'of',
  'our',
  'previous',
  'repurchase',
  'program',
  ',',
  'we',
  'repurchased',
  'approximately',
  '8.3',
  'million',
  'shares',
  'for',
  '$',
  '336',
  'million',
  'under',
  'this',
  'authorization',
  '.'],
 ['during',
  'the',
  'year',
  'ended',
  'october',
  '31',
  ',',
  '2017',
  'we',
  'repurchased',
  'approximately',
  '4.1',
  'million',
  'shares',
  'for',
  '$',
  '194',
  'million',
  'under',
  'this',
  'authorization',
  '.'],
 ['as',
  'of',
  'october',
  '31',
  ',',
  '2017',
  ',',
  'we',
  'had',
  'remaining',
  'authorization',
  'to',
  'repurchase',
  'up',
  'to',
  '$',
  '610',
  'million',
  'of',
  'our',
  'common',
  'stock',
  'under',
  'this',
  'program']]

In [97]:
for ind, sent in enumerate(test_p_sent_bow):
    test_p_sent_bow[ind] = [word for word in sent if word not in stopwords.words('english')]

In [98]:
test_p_sent_bow

[['table',
  'contents',
  '2016',
  ',',
  'upon',
  'completion',
  'previous',
  'repurchase',
  'program',
  ',',
  'repurchased',
  'approximately',
  '8.3',
  'million',
  'shares',
  '$',
  '336',
  'million',
  'authorization',
  '.'],
 ['year',
  'ended',
  'october',
  '31',
  ',',
  '2017',
  'repurchased',
  'approximately',
  '4.1',
  'million',
  'shares',
  '$',
  '194',
  'million',
  'authorization',
  '.'],
 ['october',
  '31',
  ',',
  '2017',
  ',',
  'remaining',
  'authorization',
  'repurchase',
  '$',
  '610',
  'million',
  'common',
  'stock',
  'program']]

In [104]:
ps = PorterStemmer()

In [105]:
for ind, sent in enumerate(test_p_sent_bow):
    test_p_sent_bow[ind] = [ps.stem(word) for word in sent]

In [106]:
test_p_sent_bow

[['tabl',
  'content',
  '2016',
  ',',
  'upon',
  'complet',
  'previou',
  'repurchas',
  'program',
  ',',
  'repurchas',
  'approxim',
  '8.3',
  'million',
  'share',
  '$',
  '336',
  'million',
  'author',
  '.'],
 ['year',
  'end',
  'octob',
  '31',
  ',',
  '2017',
  'repurchas',
  'approxim',
  '4.1',
  'million',
  'share',
  '$',
  '194',
  'million',
  'author',
  '.'],
 ['octob',
  '31',
  ',',
  '2017',
  ',',
  'remain',
  'author',
  'repurchas',
  '$',
  '610',
  'million',
  'common',
  'stock',
  'program']]

In [107]:
possible_important_words = ["remain"]

In [108]:
months = ["january", "febuary", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

In [112]:
stem_months = [ps.stem(word) for word in months]

In [109]:
units = ["hundred", "thousand", "million", "billion", "trillion"]

In [110]:
# This is how to check to see if word in list is in another list
# any(word in test_p_sent_bow[2] for word in possible_important_words)

In [114]:
for sent in test_p_sent_bow:
    if any(word in sent for word in possible_important_words):
        print(set(possible_important_words) & set(sent))
        for ind, token in enumerate(sent):
            if token in stem_months:
                print("time: ", sent[ind:ind+4])
            elif token == "$":
                print("amount: ", sent[ind+1])#, sent[ind+2])
            elif token in units:
                print("units: ", sent[ind])

{'remain'}
time:  ['octob', '31', ',', '2017']
amount:  610
units:  million


In [77]:
unique_paragraphs = paragraphs.paragraph_text.unique()
len(unique_paragraphs)

909

In [79]:
unique_paragraphs

array(['On May 28, 2015 we  announced that our board  of directors had approved a  new share repurchase program  (the   "2015 repurchase program"). The 2015 share repurchase program authorizes the purchase of up to $1.14  billion   of our common stock at the company\'s discretion  through and including November 1, 2018. The 2015  repurchase   program does  not require  the company  to  acquire a  specific number  of  shares and  may be  suspended  or   discontinued at any time. During the year ended October 31,                                                        95',
       'Table of Contents   2016, upon the completion of our previous repurchase program, we repurchased approximately 8.3 million shares   for $336  million  under  this  authorization.  During  the  year  ended  October  31,  2017  we  repurchased   approximately 4.1 million shares for $194  million under this authorization. As  of October 31, 2017, we  had   remaining authorization to repurchase up to $610 million of

In [78]:
p_with_remaining = list()
for p in unique_paragraphs:
    p_token = word_tokenize(p)
    if "remaining" in p_token:
        p_with_remaining.append(p)
len(p_with_remaining)

151

In [74]:
remaining_sent_word_token = list()
for p in p_with_remaining:
    sent_tokens = sent_tokenize(p)
    words = list()
    for sent in sent_tokens:
        word_tokens = word_tokenize(sent)
        words.append(word_tokens)
    remaining_sent_word_token.append(words)

In [75]:
remaining_sent_word_token

[[['Table',
   'of',
   'Contents',
   '2016',
   ',',
   'upon',
   'the',
   'completion',
   'of',
   'our',
   'previous',
   'repurchase',
   'program',
   ',',
   'we',
   'repurchased',
   'approximately',
   '8.3',
   'million',
   'shares',
   'for',
   '$',
   '336',
   'million',
   'under',
   'this',
   'authorization',
   '.'],
  ['During',
   'the',
   'year',
   'ended',
   'October',
   '31',
   ',',
   '2017',
   'we',
   'repurchased',
   'approximately',
   '4.1',
   'million',
   'shares',
   'for',
   '$',
   '194',
   'million',
   'under',
   'this',
   'authorization',
   '.'],
  ['As',
   'of',
   'October',
   '31',
   ',',
   '2017',
   ',',
   'we',
   'had',
   'remaining',
   'authorization',
   'to',
   'repurchase',
   'up',
   'to',
   '$',
   '610',
   'million',
   'of',
   'our',
   'common',
   'stock',
   'under',
   'this',
   'program']],
 [['The',
   'Company',
   "'s",
   'stock',
   'repurchase',
   'program',
   'allows',
   'it',
   'to',
 

In [76]:
for p in remaining_sent_word_token:
    for sent in p:
        if any(word in sent for word in possible_important_words):
            print(set(possible_important_words) & set(sent))
            for ind, token in enumerate(sent):
                if token in months:
                    print("month: ", sent[ind:ind+4])
                elif token == "$":
                    print("amount: ", sent[ind+1])#, sent[ind+2])
                elif token in units:
                    print("units: ", sent[ind])

{'remaining'}
month:  October
amount:  610
units:  million
{'remaining'}
amount:  415.1
units:  million
month:  December
{'remaining'}
month:  May
month:  August
amount:  750
units:  million
{'remaining'}
month:  September
amount:  788.9
units:  million
{'remaining'}
month:  December
amount:  446.3
units:  million
{'remaining'}
amount:  4,000
amount:  2,762
amount:  -
amount:  -
month:  November
amount:  3,845
amount:  -
amount:  296
month:  December
amount:  1,238
amount:  1,083
amount:  1,083
{'remaining'}
month:  December
amount:  1.2
units:  billion
{'remaining'}
month:  December
units:  million
{'remaining'}
month:  December
amount:  389.5
units:  million
amount:  293.4
units:  million
month:  December
month:  November
{'remaining'}
month:  December
amount:  200.0
units:  million
month:  April
{'remaining'}
month:  December
amount:  200.0
units:  million
month:  April
{'remaining'}
month:  October
month:  October
amount:  67.72
month:  November
month:  November
month:  December
mo

In [82]:
"Share Repurchase Program   In May 2017, the Company's Board of Directors increased the share repurchase authorization from $175  billion   to $210 billion of the Company's  common stock, of which $166 billion  had been utilized as of September  30,   2017. The Company's share repurchase program does not  obligate it to acquire any specific number of  shares"

"Share Repurchase Program   In May 2017, the Company's Board of Directors increased the share repurchase authorization from $175  billion   to $210 billion of the Company's  common stock, of which $166 billion  had been utilized as of September  30,   2017. The Company's share repurchase program does not  obligate it to acquire any specific number of  shares"

In [83]:
"On February 15, 2018, AbbVie announced that its board of directors declared an increase in the company's   quarterly cash dividend from $0.71 per share to $0.96 per share beginning with the dividend payable on May   15, 2018 to stockholders of record as of April 13, 2018"

"On February 15, 2018, AbbVie announced that its board of directors declared an increase in the company's   quarterly cash dividend from $0.71 per share to $0.96 per share beginning with the dividend payable on May   15, 2018 to stockholders of record as of April 13, 2018"

In [84]:
'Stock Repurchase Program   To facilitate  our stock  repurchase program,  designed  to return  value to  our stockholders  and  minimize   dilution from  stock  issuances, we  may  repurchase shares  in  the open  market  or enter  into  structured   repurchase agreements with  third parties.  In January  2017, our  Board of  Directors approved  a new  stock   repurchase program granting us authority to repurchase up to $2.5 billion in common stock through the end  of   fiscal 2019. The new stock repurchase program approved by  our Board of Directors is similar to our  previous   stock repurchase programs'

'Stock Repurchase Program   To facilitate  our stock  repurchase program,  designed  to return  value to  our stockholders  and  minimize   dilution from  stock  issuances, we  may  repurchase shares  in  the open  market  or enter  into  structured   repurchase agreements with  third parties.  In January  2017, our  Board of  Directors approved  a new  stock   repurchase program granting us authority to repurchase up to $2.5 billion in common stock through the end  of   fiscal 2019. The new stock repurchase program approved by  our Board of Directors is similar to our  previous   stock repurchase programs'

In [85]:
"Common Stock Repurchases   The Company's common stock repurchase program has been in place since August 2004. In the aggregate, the   Board of Directors has authorized the Company to repurchase $6.2 billion of the Company's common stock under   the program. The Company may repurchase outstanding shares of its common stock from time to time in the open   market and through privately negotiated transactions. Unless terminated earlier by resolution of the   Company's Board of Directors, the repurchase program will expire when the Company has repurchased all shares   authorized under the program. As of October 28, 2017, the Company had repurchased a total of approximately   147.0 million shares of its common stock for approximately $5.4 billion under this program. An additional   $792.5 million remains available for repurchase of shares under the current authorized program. The   repurchased shares are held as authorized but unissued shares of common stock. In connection with the   Acquisition, the Company temporarily suspended the common stock repurchase plan"

"Common Stock Repurchases   The Company's common stock repurchase program has been in place since August 2004. In the aggregate, the   Board of Directors has authorized the Company to repurchase $6.2 billion of the Company's common stock under   the program. The Company may repurchase outstanding shares of its common stock from time to time in the open   market and through privately negotiated transactions. Unless terminated earlier by resolution of the   Company's Board of Directors, the repurchase program will expire when the Company has repurchased all shares   authorized under the program. As of October 28, 2017, the Company had repurchased a total of approximately   147.0 million shares of its common stock for approximately $5.4 billion under this program. An additional   $792.5 million remains available for repurchase of shares under the current authorized program. The   repurchased shares are held as authorized but unissued shares of common stock. In connection with the   Acquis

In [86]:
'Issuer Purchases of Equity Securities   Autodesk\'s stock repurchase program is largely to help offset the dilution from the issuance of stock under   our employee stock plans and for such other purposes as may be in the interests of Autodesk and its   stockholders, and has the effect of returning excess cash generated from our business to stockholders. The   share repurchase program does not have an expiration date and the pace and timing of repurchases will depend   on factors such as cash generation from operations, available surplus, the volume of employee stock plan   activity, cash requirements for acquisitions, economic and market conditions, stock price and legal and   regulatory requirements. In September 2016, the Board of Directors approved a plan which authorized the   repurchase of up to an additional 30.0 million shares of the Company\'s common stock. As of January 31, 2018,   10.4 million shares have been repurchased under this plan. During the three and twelve months ended January   31, 2018, we repurchased 2.5 million and 6.9 million shares, respectively, of our common stock under the   Board of Director authorized share repurchase program. At January 31, 2018, 19.6 million shares remained   available for repurchase under the repurchase program approved by the Board of Directors. See Note 9,   "Stockholders\' (Deficit) Equity (Deficit)," in the Notes to Consolidated Financial Statements for further   discussion'

'Issuer Purchases of Equity Securities   Autodesk\'s stock repurchase program is largely to help offset the dilution from the issuance of stock under   our employee stock plans and for such other purposes as may be in the interests of Autodesk and its   stockholders, and has the effect of returning excess cash generated from our business to stockholders. The   share repurchase program does not have an expiration date and the pace and timing of repurchases will depend   on factors such as cash generation from operations, available surplus, the volume of employee stock plan   activity, cash requirements for acquisitions, economic and market conditions, stock price and legal and   regulatory requirements. In September 2016, the Board of Directors approved a plan which authorized the   repurchase of up to an additional 30.0 million shares of the Company\'s common stock. As of January 31, 2018,   10.4 million shares have been repurchased under this plan. During the three and twelve months e

In [115]:
A = [[9, 5, 6], [1, 2, 3]]

sum(A, [])

[9, 5, 6, 1, 2, 3]